In [ ]:
from sys import exit

# clean output directory every time
import shutil
try: shutil.rmtree('out')
except FileNotFoundError: pass

from pathlib import Path
Path('out').mkdir(parents=True, exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm

In [ ]:
# data files are exactly as received from MIRA Helsinki Study. This code does not write to these files.
data_master = 'data/mira/MIRA Master file 180527.sav'
data_lab = 'data/mira/HUSLAB Data final_WithMixedVegans.txt'
data_intakes = 'data/mira/muuttujat_analyysiin.txt'
data_thl = 'data/mira/growth-curves.tsv'
data_questionnaire = 'data/mira/data huoltajan tausta 171106.sav'
data_food_record = 'data/mira/radata.txt'

In [ ]:
#declare common column names as variables
ldl = 'fP-Kol-LDL (mmol/l)'
tc = 'fP-Kol (mmol/l)'
serum_lipids = [
    tc,
    ldl, 
    'fP-Kol-HDL (mmol/l)', 
    'fP-Trigly (mmol/l)'
]

In [ ]:
# read in mira study data
lab_results = pd.read_csv(data_lab, sep='\t', decimal=",")
intakes = pd.read_csv(data_intakes, sep='\t')

subjects_all = intakes.merge(lab_results, how='left', on='ID')

In [ ]:
# classify diets
g6_map = {
    'Pesco-vegetarian': 'Vegetarian',
    'Vegan': 'Vegan',
    'Control': 'Omnivore',
    'Control (vegan in daycare)': 'Omnivore',
    'Vegetarian': 'Vegetarian',
}
diet_class = 'diet classification'
diet_classes = ['Vegan','Vegetarian','Omnivore']

s = subjects_all.Group4.map(g6_map)
s = s.fillna('Omnivore')

subjects_all.insert(6,diet_class,s)

In [ ]:
#one participant was reported to be vegan, but had consumed dairy products
subjects_all.loc[subjects_all.ID==402,diet_class] = 'Vegetarian'

In [ ]:
subjects_all[['ID','Group2','Group3','Group4',diet_class]].to_csv('out/subject_grouping.csv', index=False)

In [ ]:
# Use Finnish THL curves for child BMI SDS

curves = pd.read_csv(data_thl,sep='\t',decimal=',')
curves.columns = [c.lower() for c in curves.columns]

# no curve for children under two, fill in with a linear extrapolation

def fill_start_with_linear_extrapolation(s):
    i = s.first_valid_index()
    x1 = s.loc[i]
    x2 = s.loc[2*i]
    x0 = x1 - (x2-x1)
    return pd.concat(
        [
            pd.Series(np.linspace(x0,x1,i)),
            s[i:]
        ]
    )


for c in 'bmi_mean_m','bmi_sd_m','bmi_nu_m','bmi_mean_f','bmi_sd_f','bmi_nu_f':
    s = curves[c]
    s2 = fill_start_with_linear_extrapolation(s)
    curves.insert(
        curves.columns.get_loc(c)+1,
        c+'_filled',
        s2
    )

# BMISDS = ((BMIlaskettu / muBMI) ^ nuBMI – 1) / (nuBMI × sigmaBMI)
def translate_sex(s):
    if s in ('M','m'): return 'm'
    if s in ('N','n','F','f'): return 'f'
    return None

def bmi_sds(weight,height,age,sex):
    # NaN check
    if age != age: return None
    
    age = round(age,2)
    sex = translate_sex(sex)

    bmi = weight/height**2

    row = curves[curves.age==age].iloc[0]
    mu_bmi = row['bmi_mean_'+sex+'_filled']
    nu_bmi = row['bmi_nu_'+sex+'_filled']
    sigma_bmi = row['bmi_sd_'+sex+'_filled']

    bmi_sds = ((bmi/mu_bmi)**nu_bmi - 1) / (nu_bmi * sigma_bmi)

    return bmi_sds

bmi_sds = subjects_all.apply(
    lambda row: bmi_sds(
        row.Weight,
        row.Height/100,
        row.AntAge, #row.Bage,
        row.Sex
    ),
    axis=1
)
subjects_all.insert(12,'bmi_sds',bmi_sds)

In [ ]:
# classify BMI by SDS

def is_female(s):
    return s in ('N','n','F','f')

def is_male(s):
    return s in ('M','m')

def classify_bmi_sds(bmi_sds,sex):
    if is_female(sex):
        if bmi_sds < -2.2187: return 'Significantly underweight'
        if bmi_sds < -1.6482: return 'Underweight'
        if bmi_sds > 2.7600: return 'Severely Obese'
        if bmi_sds > 2.1065: return 'Obese'
        if bmi_sds > 1.1629: return 'Overweight'
        return 'Normal'
    if is_male(sex):
        if bmi_sds < -2.3456: return 'Significantly underweight'
        if bmi_sds < -1.8344: return 'Underweight'
        if bmi_sds > 2.3600: return 'Severely Obese'
        if bmi_sds > 1.7016: return 'Obese'
        if bmi_sds > 0.7784: return 'Overweight'
        return 'Normal'
    return None

bmi_class = subjects_all.apply(
    lambda row: classify_bmi_sds(
        row.bmi_sds,
        row.Sex
    ),
    axis=1
)

subjects_all.insert(13,'bmi_class',bmi_class)

In [ ]:
subjects_all['ENER_kcal_per_d'] = subjects_all.ENERJ_per_d / 4.184

In [ ]:
# this file contains the weight proportion of animal source ingredients in a food item/dish as calculated or estimated by a researcher
# links to Fineli food items for foods recorded using Fineli codes
data_food_animal_proportion = 'data/food-animal-proportion.csv'

In [ ]:
food_records = pd.read_csv(
    data_food_record, 
    sep='\t', 
    encoding='iso-8859-1')

food_animal_percent = pd.read_csv(
    data_food_animal_proportion, 
    index_col='code'
).drop(columns=['link'])

food_records = food_records.merge(
    food_animal_percent, 
    left_on='Code', 
    right_index=True, 
    how='left'
)
food_records['timestamp'] = pd.to_datetime(
    food_records.DaDate + ' ' + food_records.MaTime,
    format='%d.%m.%Y %H:%M:%S'
)

In [ ]:
#merge diet grouping to food records for checks
food_records = food_records.merge(
    subjects_all[['ID',diet_class]],
    on='ID', 
    how='left'
)
fltr_is_vegan = (food_records[diet_class] == 'Vegan')

In [ ]:
missing_ap = food_records.animal_proportion.isna()
if missing_ap.any():
    display(food_records[missing_ap])
    exit()

In [ ]:
#calculate age when food records taken
master = pd.read_spss(data_master)

df = pd.DataFrame()
df['dob'] = pd.to_datetime(
    master.set_index('ID').syntymaaika
)
df['date_of_first_food_record'] = pd.to_datetime(
    food_records.groupby('ID').DaDate.min(),
    dayfirst=True
)
df['age_at_first_food_record'] = df.date_of_first_food_record - df.dob

subjects_all = subjects_all.merge(
    df.drop(columns=['dob']).reset_index()
)

In [ ]:
# global settings for graph output
scale=10
sns.set_theme(style='white',font_scale=2)

In [ ]:
#sanity checking food record lenths
df = food_records.groupby('ID').agg({'timestamp': ['min', 'max']})
df.columns = ['ts_min','ts_max']
df = df.merge(
    food_records[['ID','DaDate']].groupby('ID').nunique(),
    on='ID'
)
df.rename(columns = {'DaDate':'record_distinct_dates'}, inplace = True)
df.record_distinct_dates.hist(range=(0,6),bins=6)
df.to_csv('out/fr_days.csv')

In [ ]:
# sanity check which food items with nonzero animal content appear for vegans

def vegan_animal_consumption():    
    return food_records[
        (food_records.animal_proportion > 0) 
        & fltr_is_vegan
    ].groupby(['name','MaName']).count()[diet_class]    

df = vegan_animal_consumption()
df.to_csv('out/vegan_animal_consumption.csv')
df

In [ ]:
# Average cooking fat present in one recipe (in soy bolognese, Fineli code 28961, in fried onion, Fineli code 3230) 
# was manually confirmed from the food record to be incorrect
# Participant had consumed a sauce that does not contain added fat, unlike the generic sauce which was initially entered. 
# Average cooking oil was dropped from these two occasions where it had been erroneously entered.

fltr = (food_records.name=='Ruoanvalmistusrasva keskiarvo') & (food_records[diet_class]=='Vegan')
rows_to_drop = food_records[fltr]
idx_to_drop = rows_to_drop.index
food_records = food_records.drop(idx_to_drop)

In [ ]:
def zero_ap_for_vegans(ingredient_name):
    
    fltr = (food_records.name == ingredient_name) & fltr_is_vegan

    food_records.loc[fltr,'animal_proportion'] = 0
    food_records.loc[fltr,'animal_decile'] = 0

    print(f"{sum(fltr)} food records of vegans consuming '{ingredient_name}' updated to animal_proportion==0")

# Correcting data entry which was confirmed to be a typo: vegetable stock was entered as fish stock.

zero_ap_for_vegans('Knorr Kalaliemi, jauhe, vähäsuolainen')

In [ ]:
vegan_animal_consumption()

In [ ]:
# calculate ASE proportion
r_animal = 'r_animal_source_energy'
r_animal_label = 'Animal source energy proportion'

e = food_records[['ID','ENERJ','animal_proportion']]
e = e.assign(ENERJ_animal=(e.ENERJ * e.animal_proportion)).drop(columns=['animal_proportion'])

#energy per subject
eps = e.groupby(['ID']).sum()
eps = eps.assign(r_animal_source_energy=(eps.ENERJ_animal / eps.ENERJ))

eps.to_csv('out/energy_intake_per_participant.csv')

subjects_all = subjects_all.merge(eps, on='ID')

In [ ]:
# confirming zero animal source foods recorded for vegans

vegans_nonzero_asep = subjects_all[
    (subjects_all[diet_class] == 'Vegan') &
    (subjects_all[r_animal] > 0)
][r_animal]


vegans_nonzero_asep = pd.concat(
    [
        vegans_nonzero_asep,
        vegans_nonzero_asep.describe()
    ]
)

# vegans_nonzero_asep.to_csv('out/vegans_nonzero_asep.csv')
display(vegans_nonzero_asep)

# Subject selection

In [ ]:
# data has been manually verified
fr_too_short = (105, 405, 801, 802, 119)
fr_incomplete_day = (401,)
fr_no_weekend = (404,)
fr_ill_during_study = (702,)
fr_invalid = fr_too_short + fr_incomplete_day + fr_no_weekend + fr_ill_during_study

subjects_fr = subjects_all[~subjects_all.ID.isin(fr_invalid)]
subjects_ldl = subjects_fr[subjects_fr[ldl].notna()]

## Figure 1

Distribution of the ASE proportions by the diet classification (grouping based on the food records and background questionnaires). Participants' dietary classification is indicated with color coding, with green for vegan, orange for vegetarian and blue for omnivore group. The vegetarian group included lactovegetarians, lacto-ovo-vegetarians and pescovegetarians.

In [ ]:
sns.set_context("paper")

tricolor_palette=['#55a868','#dd8452','#4c72b0']

plt.close()
fg = sns.displot(
    subjects_fr,
    x=r_animal,
    binwidth=0.05,
    hue=diet_class,
    hue_order=diet_classes,
    multiple='stack',
    height=scale,
    palette=tricolor_palette
)
fg.axes[0,0].set_xlabel(r_animal_label)
fg.axes[0,0].set_ylabel('Number of participants')

plt.savefig('out/Fig1.svg')
plt.savefig('out/Fig1.png')

# Table 1
Describing data with median (min-max).

In [ ]:
intakes = [
    'ENERJ_per_d',
    'ENER_kcal_per_d',
    'FAT_e_pros',
    'FASAT_e_pros',
    'FAMS_e_pros',
    'FAPU_e_pros',
    'CHOL_per_MJ',
    'CHOL_per_d',
    'PROT_e_pros',
    'CHO_e_pros',
    'SUCS_e_pros',
    'FIBC_per_MJ',
    'FIBC_per_d',
    'SALT_per_MJ',
    'SALT_per_d',
    'FOL_per_d'
]
biomarkers = [
    'fP-Kol (mmol/l)',
    'fP-Kol-LDL (mmol/l)',
    'fP-Kol-HDL (mmol/l)',
    'fP-Trigly (mmol/l)',
    'fE-Folaat (nmol/l)'    
]

In [ ]:
def summarize(name, df):
    count_diet_classes = df[diet_class].value_counts().to_frame()
    count_sexes = df.sukupuoli.value_counts().to_frame()
    count_bmi_class = df.bmi_class.value_counts().to_frame()
    desc_intakes = df[['age_at_first_food_record','bmi_sds']+intakes+biomarkers].describe().transpose()[['50%','min','max']]
    
    with pd.ExcelWriter(
        f'out/summary_{name}.xlsx',
        mode='w'
    ) as writer:
        
        count_diet_classes.to_excel(
            writer,
            sheet_name='diet_classes'
        )
    
        count_sexes.to_excel(
            writer,
            sheet_name='sexes'
        )

        count_bmi_class.to_excel(
            writer,
            sheet_name='bmi_class'
        )

        desc_intakes.to_excel(
            writer,
            sheet_name='intakes'
        )    
    
    print(len(df.index))
    display(count_diet_classes)
    display(count_sexes)
    display(count_bmi_class)
    display(desc_intakes)

In [ ]:
summarize('participants_with_valid_fr', subjects_fr)

In [ ]:
summarize('participants_with_valid_fr_and_blood_sample',subjects_ldl)

# Table 2

In [ ]:
def correlation_values(subjects, xs,ys):
    table = []
    for x in xs:
        for y in ys:
            df = subjects[[x,y]].dropna()
            n = len(df.index)
            pr, pp = pearsonr(df[x],df[y])
            sr, sp = spearmanr(df[x],df[y])
            table.append([n,x,y,pr,pp,sr,sp])

    df = pd.DataFrame(table,columns=['n','x','y','pearson_r','pearson_p','spearman_r','spearman_p'])
    df['pearson_bh_0_05'] = multipletests(df['pearson_p'], alpha=0.05, method='fdr_bh')[0]
    df['spearman_bh_0_05'] = multipletests(df['spearman_p'], alpha=0.05, method='fdr_bh')[0]
    df = df.sort_values(by='pearson_p')
    df = df.round(3)
    return df

In [ ]:
df = correlation_values(
    subjects_ldl,
    [r_animal],
    intakes
)
df.to_csv('out/table2-correlations-ASEP-intakes.csv', index=False)
df

# Figure 3

In [ ]:
from string import ascii_lowercase

markers=['o','s','^']

def regplot_r_animal(df,ax,y,y_label,set_xlabel=True):
    
    sample = df[['r_animal_source_energy',diet_class,y]].dropna()
    
    right = sample.r_animal_source_energy.max()
    ax.set_xlim(-0.01, right+0.01)
    
    
    #sns.regplot(ax=ax, x=r_animal, y=y, scatter=False, data=df)
    sns.scatterplot(
        ax=ax, 
        x=r_animal, 
        y=y, 
        hue=diet_class,
        hue_order=diet_classes,
        style=diet_class,
        markers=markers,
        data=sample, 
        s=30*scale,
        palette=tricolor_palette,
        legend=False
    )
    if set_xlabel:
        ax.set_xlabel(r_animal_label)
    else:
        ax.set_xlabel(None)
    ax.set_ylabel(y_label)
    
    sr, sp = spearmanr(sample.r_animal_source_energy,sample[y])
    
    label = f'ρ= {sr:.3f}'
    if sr < 0:
        ax.text(0.7,0.9, label, transform=ax.transAxes)
    elif sr > 0:
        ax.text(0.1,0.9, label, transform=ax.transAxes)


from matplotlib.lines import Line2D
def splats(n):
    return [
        Line2D(
            [0], 
            [0], 
            marker=markers[i],
            color='w',
            markerfacecolor=tricolor_palette[i],
            markersize=15
        )
        for i in range(0,n)
    ]


def fig_of_regplots(df,ys,cols):

    plt.close()
    sns.set_theme(style='white',font_scale=2)
    
    rows = int(len(ys)/cols)+(len(ys)%cols > 0)
    h = rows*scale
    if rows > 1: h+=2
    w = cols*scale+2
    
    fig, axs = plt.subplots(rows,cols,figsize=(w,h))
        
    if rows > 1:
        axs_flat = [ax for row in axs for ax in row]
    else:
        axs_flat = axs

    l = list(zip(ys.keys(),ys.values(),axs_flat))
    i = 0
    for t in l:
        ax = t[2]
        y = t[0]
        regplot_r_animal(df,ax,y,t[1],set_xlabel=False)
        i+=1

    for ax in axs_flat[len(ys):]:
        fig.delaxes(ax)
    
    if rows*cols > len(ys):
        plt.figlegend(splats(3),diet_classes,bbox_to_anchor=((1.09-1/cols),1.03/rows))        
    else:
        plt.figlegend(splats(3),diet_classes)

    
    return fig, axs

In [ ]:
sns.set_context("paper")

ys = {
    'PROT_e_pros': 'Protein intake (E%)',
    'FIBC_per_MJ': 'Fiber intake (g/MJ)',
    'FOL_per_d': 'Folate intake (µg/d)',
    'FASAT_e_pros': 'Saturated Fat intake (E%)',
    'FAPU_e_pros': 'Polyunsaturated Fat intake (E%)',
    'CHOL_per_MJ': 'Cholesterol intake (mg/MJ)',
    'fP-Kol-LDL (mmol/l)': 'LDL cholesterol intake (mmol/l)',
    'fE-Folaat (nmol/l)': 'Erythrocyte folate intake (nmol/l)'  
}

fig, axs = fig_of_regplots(subjects_fr,ys,3)

plt.savefig('out/Fig3.svg')
plt.savefig('out/Fig3.png')

plt.show()

# Readable versions of source data
Write excel files to /out

In [ ]:
subjects_all.to_excel('out/subjects_all.xlsx')
food_records.to_excel('out/food_records.xlsx')

# Exploration